In [71]:
import torch
from torch import nn
from torch import Tensor
import requests
import os
from pathlib import Path
from typing import Callable
from functools import partial


URL = 'https://download.pytorch.org/models/resnet18-5c106cde.pth'

In [72]:
from torchvision.models import resnext50_32x4d, resnext101_32x8d, resnet50, wide_resnet50_2
from glasses.nn.models.classification.resnetxt import ResNetXt, ResNetXtBottleNeckBlock
from glasses.nn.models.classification.wide_resnet import WideResNet
from glasses.nn.models.classification.resnet import ResNet


In [73]:
ResNet.resnet50()

ResNet(
  (encoder): ResNetEncoder(
    (gate): Sequential(
      (conv): Conv2dPad(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
      (bn): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (act): ReLU(inplace=True)
      (pool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
    )
    (blocks): ModuleList(
      (0): ResNetLayer(
        (block): Sequential(
          (0): ResNetBottleneckBlock(
            (block): ResidualAdd(
              (block): Sequential(
                (conv1): Conv2dPad(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
                (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
                (act1): ReLU(inplace=True)
                (conv2): Conv2dPad(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
                (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=Tr

In [74]:
%load_ext autoreload

%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [75]:
from glasses.nn.models.classification.efficientnet import EfficientNet
from torchsummary import summary
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

summary(ResNetXt.resnext50_32x4d().to(device), (3, 224, 224))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
         Conv2dPad-1         [-1, 64, 112, 112]           9,408
       BatchNorm2d-2         [-1, 64, 112, 112]             128
              ReLU-3         [-1, 64, 112, 112]               0
         MaxPool2d-4           [-1, 64, 56, 56]               0
         Conv2dPad-5          [-1, 128, 56, 56]           8,192
       BatchNorm2d-6          [-1, 128, 56, 56]             256
              ReLU-7          [-1, 128, 56, 56]               0
         Conv2dPad-8          [-1, 128, 56, 56]           4,608
       BatchNorm2d-9          [-1, 128, 56, 56]             256
             ReLU-10          [-1, 128, 56, 56]               0
        Conv2dPad-11          [-1, 256, 56, 56]          32,768
      BatchNorm2d-12          [-1, 256, 56, 56]             512
        Conv2dPad-13          [-1, 256, 56, 56]          16,384
      BatchNorm2d-14          [-1, 256,

(tensor(25028904), tensor(25028904))

In [44]:
summary(wide_resnet50_2(False).to(device), (3, 224, 224))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1         [-1, 64, 112, 112]           9,408
       BatchNorm2d-2         [-1, 64, 112, 112]             128
              ReLU-3         [-1, 64, 112, 112]               0
         MaxPool2d-4           [-1, 64, 56, 56]               0
            Conv2d-5          [-1, 128, 56, 56]           8,192
       BatchNorm2d-6          [-1, 128, 56, 56]             256
              ReLU-7          [-1, 128, 56, 56]               0
            Conv2d-8          [-1, 128, 56, 56]         147,456
       BatchNorm2d-9          [-1, 128, 56, 56]             256
             ReLU-10          [-1, 128, 56, 56]               0
           Conv2d-11          [-1, 256, 56, 56]          32,768
      BatchNorm2d-12          [-1, 256, 56, 56]             512
           Conv2d-13          [-1, 256, 56, 56]          16,384
      BatchNorm2d-14          [-1, 256,

(tensor(68883240), tensor(68883240))

In [71]:
from __future__ import annotations
import torch
import torch.nn.functional as F
from torch import nn
from torch import Tensor
from collections import OrderedDict
from typing import List
from functools import partial
from glasses.nn.blocks import ConvBnAct, Conv2dPad
from glasses.nn.models.segmentation.unet import UNet


In [72]:
EfficientNet.config

{'b0': (1.0, 1.0, 224, 0.2),
 'b1': (1.0, 1.1, 240, 0.2),
 'b2': (1.1, 1.2, 260, 0.3),
 'b3': (1.2, 1.4, 300, 0.3),
 'b4': (1.4, 1.8, 380, 0.4),
 'b5': (1.6, 2.2, 456, 0.4),
 'b6': (1.8, 2.6, 528, 0.5),
 'b7': (2.0, 3.1, 600, 0.5)}

In [80]:
from glasses.nn.models.classification.efficientnet import EfficientNet
from torchsummary import summary
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

model_dst  = EfficientNet.b5()

info = summary(model_dst.to(device), (3,224,224))

# model_dst

info[0].item()

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
         Conv2dPad-1         [-1, 48, 112, 112]           1,296
       BatchNorm2d-2         [-1, 48, 112, 112]              96
             Swish-3         [-1, 48, 112, 112]               0
   DepthWiseConv2d-4         [-1, 48, 112, 112]             432
       BatchNorm2d-5         [-1, 48, 112, 112]              96
             Swish-6         [-1, 48, 112, 112]               0
 AdaptiveAvgPool2d-7             [-1, 48, 1, 1]               0
            Conv2d-8             [-1, 12, 1, 1]             588
             Swish-9             [-1, 12, 1, 1]               0
           Conv2d-10             [-1, 48, 1, 1]             624
          Sigmoid-11             [-1, 48, 1, 1]               0
        ChannelSE-12         [-1, 48, 112, 112]               0
        Conv2dPad-13         [-1, 24, 112, 112]           1,152
      BatchNorm2d-14         [-1, 24, 1

30389784

In [ ]:
model_dst

In [ ]:
y_dst

tensor([[-9.3545e-01,  4.4536e-01,  4.2809e-02, -7.1374e-01,  3.6706e-01,
          2.3940e-01, -1.3259e+00, -3.2074e-01, -1.6859e+00, -7.2495e-01,
         -1.2006e+00, -5.1331e-01, -7.6875e-01, -1.4668e+00, -8.2695e-01,
         -4.0084e-02,  3.0943e-01, -7.2798e-01, -4.7880e-01, -8.8295e-01,
         -1.6215e+00,  7.9493e-01,  3.6127e-02, -5.1066e-01, -4.0636e-01,
         -8.5400e-01, -1.2117e+00, -2.1563e-01, -1.1900e+00, -5.3588e-01,
         -9.6766e-01, -8.8796e-01, -1.2135e+00, -6.0439e-01, -7.0122e-01,
         -1.4644e+00, -1.1432e+00, -9.4588e-01, -9.1610e-01, -1.1081e+00,
         -7.2389e-01, -8.2417e-01, -8.6236e-01, -1.3655e+00, -1.6005e+00,
         -1.3012e-01, -1.6388e+00, -7.9500e-01, -1.2640e+00, -1.1749e+00,
         -1.2268e+00, -4.6094e-01, -7.8901e-01, -1.4785e+00, -2.4503e+00,
         -5.8356e-01, -1.4344e+00, -6.2406e-01, -1.2202e+00, -3.8103e-01,
         -1.2667e+00, -7.2553e-01, -1.5403e+00, -2.9658e-01, -7.7501e-01,
         -4.5203e-01, -1.2975e+00, -1.

In [ ]:
model_src